## Tracing Toy example -- a simple function 

In [6]:
import inspect

In [36]:
def myfunc(a):
    if a == 1 : 
        x = 1 + 2
        return x
    elif a >1: 
        y = 3+ 4
        return y

In [37]:
myfunc(4)

7

In [72]:
import pickle as pkl
relevant_lines=[]
def profile_func( frame, event, arg):
    #Profile functions should have three arguments: frame, event, and arg. 
    #frame is the current stack frame. 
    #event is a string: 'call', 'return', 'c_call', 'c_return', or 'c_exception'.
    #arg depends on the event type.

    if event == 'line' :
        fcode = frame.f_code             
        line_no = frame.f_lineno
        func_name = fcode.co_name
        if (func_name=="myfunc"):
            print (f"{frame.f_code.co_filename}--{func_name}--{line_no}")  
#             print (fcode)
#             codelines = open(frame.f_code.co_filename).readlines()
#             code = codelines[line_no-1]
#             if not any(construct in code for construct in ignore_construct):
#                 print (f"{frame.f_code.co_filename}--{func_name}--{line_no}")  
#                 print (codelines[line_no-1])

    return profile_func

In [73]:
import sys

In [74]:
def start_tracing():
    print ("-----------start_tracing-----------")
    # Implement python source debugger: https://docs.python.org/3/library/sys.html#sys.settrace
    # setprofile faster than settrace (only go through I/O)
    sys.settrace(profile_func) 
def stop_tracing():
    print ("-----------stop_tracing-----------")
    sys.settrace(None) 

In [75]:
start_tracing()

-----------start_tracing-----------


In [76]:
myfunc(4)

<ipython-input-36-d8c4f383f18c>--myfunc--2
<ipython-input-36-d8c4f383f18c>--myfunc--5
<ipython-input-36-d8c4f383f18c>--myfunc--6
<ipython-input-36-d8c4f383f18c>--myfunc--7


7

In [78]:
stop_tracing()

-----------stop_tracing-----------


In [80]:
code = inspect.getsource(func)
print (code)

def func(a):
    if a == 1 : 
        x = 1 + 2
        return x
    elif a >1: 
        y = 3+ 4
        return y



## Trying out tracing for Lux

In [1]:
import lux
import pandas as pd

In [81]:
df = pd.read_csv("https://raw.githubusercontent.com/lux-org/lux-datasets/master/data/car.csv")
df["Year"] = pd.to_datetime(df["Year"], format='%Y')

In [82]:
import sys

In [83]:
# import pickle as pkl
# def profile_func( frame, event, arg):
#     #Profile functions should have three arguments: frame, event, and arg. 
#     #frame is the current stack frame. 
#     #event is a string: 'call', 'return', 'c_call', 'c_return', or 'c_exception'.
#     #arg depends on the event type.
#     try: 
#         if event == 'line' :
#             fcode = frame.f_code             
#             line_no = frame.f_lineno
#             func_name = fcode.co_name
#             #includeMod = ['lux/vis', 'lux/action', 'lux/vislib', 'lux/executor', 'lux/interestingness']
#             includeMod = ['lux/executor']
#             if any(x in frame.f_code.co_filename for x in includeMod):
#                 codelines = open(frame.f_code.co_filename).readlines()
#                 if (func_name!="<lambda>"): #ignore one-liner lambda functions (repeated line events)
#                     ignore_construct = ['if','elif','for','return'] # prune out these control flow programming constructs
#                     code = codelines[line_no-1]
#                     if not any(construct in code for construct in ignore_construct):
#                         print (f"{frame.f_code.co_filename}--{func_name}--{line_no}")  
#                         print (codelines[line_no-1])
# #                 print (fcode)
                
#                 #print (inspect.getsourcelines(fcode))
# #                 pkl.dump(frame.f_code,open("frame.pkl",'wb'))
#     except:
#         # pass  # just swallow errors to avoid interference with traced processes
#         raise  # for debugging
#     return profile_func

In [102]:
import pickle as pkl
relevant_lines=[]
def profile_func( frame, event, arg):
    #Profile functions should have three arguments: frame, event, and arg. 
    #frame is the current stack frame. 
    #event is a string: 'call', 'return', 'c_call', 'c_return', or 'c_exception'.
    #arg depends on the event type.
    # See: https://docs.python.org/3/library/sys.html#sys.settrace
    try: 
        if event == 'line' :
            #frame objects are described here: https://docs.python.org/3/library/inspect.html
            fcode = frame.f_code             
            line_no = frame.f_lineno
            func_name = fcode.co_name
            #includeMod = ['lux/vis', 'lux/action', 'lux/vislib', 'lux/executor', 'lux/interestingness']
            includeMod = ['lux/vislib','lux/executor']
            if any(x in frame.f_code.co_filename for x in includeMod):
                if (func_name!="<lambda>"): #ignore one-liner lambda functions (repeated line events)
                    relevant_lines.append([frame.f_code.co_filename,func_name,line_no])
#                     print (f"{frame.f_code.co_filename}--{func_name}--{line_no}")  
#                 print (fcode)
                
                #print (inspect.getsourcelines(fcode))
#                 pkl.dump(frame.f_code,open("frame.pkl",'wb'))
    except:
        # pass  # just swallow errors to avoid interference with traced processes
        raise  # for debugging
    return profile_func

In [103]:
# from black import format_str, FileMode
# res = format_str(selected, mode=FileMode())
# print(res)

In [104]:
# import glob
# glob.glob("lux/*")

In [105]:
def start_tracing():
    print ("-----------start_tracing-----------")
    # Implement python source debugger: https://docs.python.org/3/library/sys.html#sys.settrace
    # setprofile faster than settrace (only go through I/O)
    sys.settrace(profile_func) 
def stop_tracing():
    print ("-----------stop_tracing-----------")
    sys.settrace(None) 

In [106]:
start_tracing()

-----------start_tracing-----------


In [107]:
%time 
from lux.vis.Vis import Vis
vis  = Vis(["Horsepower"],df)
vis

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 35 µs


LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: BIN(Horsepower), y: COUNT(Record)) mark: histogram, score: 0.0 >

In [108]:
stop_tracing()

-----------stop_tracing-----------


In [114]:
%time
from lux.vis.Vis import Vis
vis  = Vis(["Horsepower"],df)
vis

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: BIN(Horsepower), y: COUNT(Record)) mark: histogram, score: 0.0 >

In [110]:
selected = ""
for line in relevant_lines:
    filename = line[0]
    funcname = line[1]
    line_no = line[2]-1
    if "Executor.py" not in filename or ("Executor.py" in filename and "execute" in funcname):
        codelines = open(filename).readlines()# TODO: do sharing of file content here
        if (funcname not in ['__init__']):
            code = codelines[line_no]
            ignore_construct = ['if','elif','for','return'] # prune out these control flow programming constructs                    
            ignore_lux_keyword = ['self.code','self.name','__init__','PandasExecutor.',"'''",'self.output_type']# Lux-specific keywords to ignore
            ignore = ignore_construct+ignore_lux_keyword
            if not any(construct in code for construct in ignore):
                print (f"{filename}--{funcname}--{line_no}")  
                print (codelines[line_no].lstrip())
                selected+=codelines[line_no].lstrip()

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_sampling--40
SAMPLE_START = 10000

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_sampling--41
SAMPLE_CAP = 30000

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_sampling--42
SAMPLE_FRAC = 0.75

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_sampling--58
ldf._sampled = ldf

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute--83
ldf._sampled

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute--82
vis._vis_data = (

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_filter--275
vis.data is not None

/Users/dorislee/Desktop/Research/lux/dorisjlee_fork/lux/executor/PandasExecutor.py--execute_filter--274
assert (

/Users/dorislee/Desktop/Research/

In [111]:
# We should identify root calls like this 'PandasExecutor.execute_binning(vis)' and extract their docstring as documentation to the code

In [112]:
selected = selected.replace("vis._vis_data","visdf").replace('vis.data','visdf').replace("self.data","visdf").replace('self.chart',"chart").replace('self.vis','vis')

In [113]:
print (selected)

SAMPLE_START = 10000
SAMPLE_CAP = 30000
SAMPLE_FRAC = 0.75
ldf._sampled = ldf
ldf._sampled
visdf = (
visdf is not None
assert (
filters = utils.get_filter_specs(vis._inferred_intent)
attributes = utils.get_unique_attrnames(vis._inferred_intent)
visdf = visdf[attributes]
import numpy as np
bin_attribute = list(filter(lambda x: x.bin_size != 0, vis._inferred_intent))[0]
series = visdf[
bin_attribute.attribute
series = visdf[
counts, bin_edges = np.histogram(series, bins=bin_attribute.bin_size)
bin_center = np.mean(np.vstack([bin_edges[0:-1], bin_edges[1:]]), axis=0)
visdf = pd.DataFrame(
np.array([bin_center, counts]).T,
columns=[bin_attribute.attribute, "Number of Records"],
visdf = pd.DataFrame(
visdf[attr].iloc[0], pd.Period
visdf[attr].iloc[0], pd.Interval
visdf[attr].iloc[0], pd.Period
visdf[attr].iloc[0], pd.Interval
chart = Histogram(vis)
self.tooltip = False
measure = vis.get_attr_by_data_model("measure", exclude_record=True)[0]
msr_attr = vis.get_attr_by_channel(measure.channel)

In [ ]:
# Important root calls
# ['execute_*',]

In [ ]:
print (selected)

In [ ]:
def plot_vis(vis,df):
    visdf = df
    filters = utils.get_filter_specs(vis._inferred_intent)
    attributes = utils.get_unique_attrnames(vis._inferred_intent)
    visdf = visdf[attributes]
    import numpy as np
    bin_attribute = list(filter(lambda x: x.bin_size!=0,vis._inferred_intent))[0]
    series = visdf[bin_attribute.attribute].dropna() 
    counts,bin_edges = np.histogram(series,bins=bin_attribute.bin_size)
    bin_center = np.mean(np.vstack([bin_edges[0:-1],bin_edges[1:]]), axis=0)
    visdf = pd.DataFrame(np.array([bin_center,counts]).T,columns=[bin_attribute.attribute, "Number of Records"])
    # self.output_type = output_type
    # chart = Histogram(vis)
    # vis = vis
    # visdf = vis.data
    # self.tooltip = True
    # chart = self.initialize_chart()
    # self.tooltip = False
    import altair as alt
    measure = vis.get_attr_by_data_model("measure",exclude_record=True)[0]
    msr_attr = vis.get_attr_by_channel(measure.channel)[0]
    x_min = vis.min_max[msr_attr.attribute][0]
    x_max = vis.min_max[msr_attr.attribute][1]
    x_range = abs(max(vis.data[msr_attr.attribute]) - 
    min(vis.data[msr_attr.attribute]))
    plot_range = abs(x_max - x_min)
    markbar = x_range / plot_range * 12
    chart = alt.Chart(visdf).mark_bar(size=markbar).encode(
    alt.X(msr_attr.attribute, title=f'{msr_attr.attribute} (binned)',bin=alt.Bin(binned=True), type=msr_attr.data_type, axis=alt.Axis(labelOverlap=True), scale=alt.Scale(domain=(x_min, x_max))),
    alt.Y("Number of Records", type="quantitative")
    )
    # self.encode_color()
    # color_attr = vis.get_attr_by_channel("color")
    # self.add_title()
    # chart_title = vis.title
    # self.apply_default_config()
    chart = chart.configure_title(fontWeight=500,fontSize=13,font="Helvetica Neue")
    chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont="Helvetica Neue",
    labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue",labelColor="#505050")
    chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont="Helvetica Neue",
    labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue")
    chart = chart.properties(width=160,height=150)
    # chart_dict = chart.chart.to_dict()
    return chart

In [ ]:
plot_vis(vis,df)

In [ ]:
from lux.utils import utils

In [ ]:
filters = utils.get_filter_specs(vis._inferred_intent)
attributes = utils.get_unique_attrnames(vis._inferred_intent)

In [ ]:
# code = pkl.load(open("frame.pkl",'rb'))

In [ ]:
# print(inspect.getsource(code))

In [ ]:
# print(vis.to_Altair())

### Mock Example

In [ ]:
def plot_vis(vis,df):
    # Processing Data with Altair
    visdf = df
    filters = utils.get_filter_specs(vis._inferred_intent)
    attributes = utils.get_unique_attrnames(vis._inferred_intent)
    visdf = visdf[attributes]
    import numpy as np
    bin_attribute = list(filter(lambda x: x.bin_size!=0,vis._inferred_intent))[0]
    series = visdf[bin_attribute.attribute].dropna() 
    counts,bin_edges = np.histogram(series,bins=bin_attribute.bin_size)
    bin_center = np.mean(np.vstack([bin_edges[0:-1],bin_edges[1:]]), axis=0)
    visdf = pd.DataFrame(np.array([bin_center,counts]).T,columns=[bin_attribute.attribute, "Number of Records"])

    # Plotting Histogram with Altair
    import altair as alt
    measure = vis.get_attr_by_data_model("measure",exclude_record=True)[0]
    msr_attr = vis.get_attr_by_channel(measure.channel)[0]
    x_min = vis.min_max[msr_attr.attribute][0]
    x_max = vis.min_max[msr_attr.attribute][1]
    x_range = abs(max(vis.data[msr_attr.attribute]) - 
    min(vis.data[msr_attr.attribute]))
    plot_range = abs(x_max - x_min)
    markbar = x_range / plot_range * 12
    chart = alt.Chart(visdf).mark_bar(size=markbar).encode(
    alt.X(msr_attr.attribute, title=f'{msr_attr.attribute} (binned)',bin=alt.Bin(binned=True), type=msr_attr.data_type, axis=alt.Axis(labelOverlap=True), scale=alt.Scale(domain=(x_min, x_max))),
    alt.Y("Number of Records", type="quantitative")
    )
    chart = chart.configure_title(fontWeight=500,fontSize=13,font="Helvetica Neue")
    chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont="Helvetica Neue",
    labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue",labelColor="#505050")
    chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont="Helvetica Neue",
    labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue")
    chart = chart.properties(width=160,height=150)
    return chart

In [ ]:
from lux.utils import utils
filters = utils.get_filter_specs(vis._inferred_intent)
attributes = utils.get_unique_attrnames(vis._inferred_intent)
bin_attribute = list(filter(lambda x: x.bin_size!=0,vis._inferred_intent))[0]

In [ ]:
bin_attribute

In [ ]:
#Evaluated expressions

In [ ]:
# Processing Data with Altair
visdf = df[['Horsepower']]
import numpy as np
series = visdf['Horsepower'].dropna() 
counts,bin_edges = np.histogram(series,bins=10)
bin_center = np.mean(np.vstack([bin_edges[0:-1],bin_edges[1:]]), axis=0)
visdf = pd.DataFrame(np.array([bin_center,counts]).T,columns=['Horsepower', "Number of Records"])

# Plotting Histogram with Altair
import altair as alt
measure = vis.get_attr_by_data_model("measure",exclude_record=True)[0]
msr_attr = vis.get_attr_by_channel(measure.channel)[0]
x_min = vis.min_max['Horsepower'][0]
x_max = vis.min_max['Horsepower'][1]
x_range = abs(max(vis.data['Horsepower']) - 
min(vis.data['Horsepower']))
plot_range = abs(x_max - x_min)
markbar = x_range / plot_range * 12
chart = alt.Chart(visdf).mark_bar(size=markbar).encode(
alt.X('Horsepower', title='Horsepower (binned)',bin=alt.Bin(binned=True), type=msr_attr.data_type, axis=alt.Axis(labelOverlap=True), scale=alt.Scale(domain=(x_min, x_max))),
alt.Y("Number of Records", type="quantitative")
)
chart = chart.configure_title(fontWeight=500,fontSize=13,font="Helvetica Neue")
chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont="Helvetica Neue",
labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue",labelColor="#505050")
chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont="Helvetica Neue",
labelFontWeight=400,labelFontSize=9,labelFont="Helvetica Neue")
chart = chart.properties(width=160,height=150)